# ДЗ №5, Волжина Елена
[Афинные гэпы](https://compscicenter.ru/learning/assignments/24211/)

В выравнивании вместо штрафа за гэп -d использовать штраф -p-ld (p и d задаются вами) для гэпа длины l.

In [2]:
from collections import defaultdict

test1 = True

if test1:
    # http://www.bioinf.uni-freiburg.de/Lehre/Courses/2014_SS/V_Bioinformatik_1/gap-penalty-gotoh.pdf
    M = 0            # match
    MM = -1          # mismatch
    R, S = -4, -1    # gap
    string1, string2 = "CC", "ACCT"
else:
    # http://pages.cs.wisc.edu/~bsettles/ibs08/lectures/02-alignment.pdf
    M = 1            # match
    MM = -1          # mismatch
    R, S = -3, -1    # gap
    string1, string2 = "AAT", "ACACT"

M, MM = 0, -1
R, S = -4, -1
string1, string2 = "CCAG", "ACCT"

![Algorithm](hw5_algo.jpg)

In [3]:
class Layer(object):
    MAIN = 0      # diagonal moves
    LOWER = -1    # horizontal moves
    UPPER = 1     # vertical moves
    
    CHOICES = [LOWER, UPPER, MAIN]     # order IS important
    NAMES = {LOWER: 'lower', UPPER: 'upper', MAIN: 'main'}

    
class Position(object):
    def __init__(self, layer, n_row, n_col):
        assert layer in Layer.CHOICES, "Incorrect layer!"
        assert n_row >= 0 and n_col >= 0, "Negative indices!"
        self.layer = layer
        self.n_row = n_row
        self.n_col = n_col
        self.data = (self.layer, self.n_row, self.n_col)
    
    def __str__(self):
        return str((Layer.NAMES[self.layer], self.n_row, self.n_col))
    
    def __repr__(self):
        return str(self)
    
    def __hash__(self):
        return hash(self.data)

    def __eq__(self, other):
        return self.data == other.data

    def __ne__(self, other):
        return not(self == other)
    

def align(s1, s2):
    assert s1 and s2, "one of strings is empty!"
    
    n, m = len(s1), len(s2)
    weights = defaultdict(lambda: None)
    sources = defaultdict(list)
    
    weights[Position(Layer.MAIN, 0, 0)] = 0
    
    # first row
    for n_col in range(1, m + 1):
        position = Position(Layer.MAIN, 0, n_col)
        weights[position] = R + n_col * S
        sources[position] = [Position(Layer.MAIN, 0, n_col - 1)]
        
    # first column
    for n_row in range(1, n + 1):
        position = Position(Layer.MAIN, n_row, 0)
        weights[position] = R + n_row * S
        sources[position] = [Position(Layer.MAIN, n_row - 1, 0)]

    # get possible ways to (n_row, n_col) at certain layer
    def get_ways(position):
        layer, n_row, n_col = position.layer, position.n_row, position.n_col
        if layer == Layer.MAIN:       # diagonal
            diff = M if s1[n_row - 1] == s2[n_col - 1] else MM
            return [
                (Position(Layer.MAIN, n_row - 1, n_col - 1), diff),
                (Position(Layer.UPPER, n_row, n_col), 0),
                (Position(Layer.LOWER, n_row, n_col), 0),
            ]
        elif layer == Layer.LOWER:   # horizontal
            return [
                (Position(Layer.LOWER, n_row, n_col - 1), S),
                (Position(Layer.MAIN, n_row, n_col - 1), (R + S)),
            ]
        else:                       # vertical
            return [
                (Position(Layer.UPPER, n_row - 1, n_col), S),
                (Position(Layer.MAIN, n_row - 1, n_col), (R + S)),
            ]

    # dynamically fill the rest
    def process(n_row, n_col):
        DEBUG = False
        for layer in Layer.CHOICES:
            position = Position(layer, n_row, n_col)
            for source, diff in get_ways(position):
                source_weight = weights[source]
                if source_weight is None:
                    continue
                    
                new_weight = source_weight + diff
                current_weight = weights[position]
                
                if current_weight is None or current_weight < new_weight:
                    weights[position] = new_weight
                    sources[position] = [source]
                elif current_weight == new_weight:
                    sources[position].append(source)

                if DEBUG:
                    print(position, 'from', source, source_weight, '+', diff, '=', new_weight)
            if DEBUG: print('-' * 40)
    
    for k in range(1, min(n, m) + 1):
        process(k, k)
        for n_row in range(k + 1, n + 1):
            process(n_row, k)
        for n_col in range(k + 1, m + 1):
            process(k, n_col)
        
    return weights, sources

In [4]:
alignment_weights, alignment_sources = align(string1, string2)

In [5]:
print("WEIGHTS MATRIX")
for layer in Layer.CHOICES:
    print("Layer:", Layer.NAMES[layer])
    values = [[alignment_weights[Position(layer, n_row, n_col)] for n_col in range(len(string2) + 1)]
               for n_row in range(len(string1) + 1)]
    print("\n".join(" ".join(("{: >3}".format(x) if x is not None else " --") for x in row) 
                    for row in values))

WEIGHTS MATRIX
Layer: lower
 --  --  --  --  --
 -- -10  -6  -7  -8
 -- -11 -11  -6  -7
 -- -12 -11 -11  -7
 -- -13 -13 -12 -12
Layer: upper
 --  --  --  --  --
 -- -10 -11 -12 -13
 --  -6 -10 -11 -13
 --  -7  -6 -10 -12
 --  -8  -7  -7 -11
Layer: main
  0  -5  -6  -7  -8
 -5  -1  -5  -6  -8
 -6  -6  -1  -5  -7
 -7  -6  -6  -2  -6
 -8  -8  -7  -7  -3


In [6]:
def restore_paths(sources, source):
    n_row, n_col = source.n_row, source.n_col
    if sources[source]:
        result = []
        for old_source in sources[source]:
            if source != old_source:            
                paths = restore_paths(sources, old_source)
                result.extend([p + [source] for p in paths])
            else:
                print(source, old_source)
        return result
    else:
        return [[source]]


alignment_paths = restore_paths(alignment_sources, Position(Layer.MAIN, len(string1), len(string2)))

In [6]:
def print_alignments(paths):
    for path in paths:
        res1, res2 = [], []
        for source, target in zip(path, path[1:]):
            r1, c1, r2, c2 = source.n_row, source.n_col, target.n_row, target.n_col
            if r1 + 1 == r2 and c1 + 1 == c2:
                # mis(match)
                res1.append(string1[r2 - 1])
                res2.append(string2[c2 - 1])
            elif r1 + 1 == r2 and c1 == c2:
                # down
                res1.append(string1[r2 - 1])
                res2.append("-")
            elif r1 == r2 and c1 + 1 == c2:
                # right
                res1.append("-")
                res2.append(string2[c2 - 1])
        print(res1, "\n", res2, "\n\n", sep="")

print("GLOBAL ALIGNMENTS:")
print_alignments(alignment_paths)

GLOBAL ALIGNMENTS:
['-', '-', 'C', 'C']
['A', 'C', 'C', 'T']


['C', 'C', '-', '-']
['A', 'C', 'C', 'T']


